In [3]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List, Dict
from collections import defaultdict
import nest_asyncio
nest_asyncio.apply()

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

import time
import tabulate
from qubx import lookup, logger
from qubx.core.basics import Position, ZERO_COSTS
from qubx.core.loggers import PositionsDumper, LogsWriter
from qubx.core.series import time_as_nsec
from qubx.core.utils import time_to_str, time_delta_to_str, recognize_timeframe
from qubx.utils.misc import makedirs 

 >  [dev] installing cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. 0.0.1
                   ╹       



In [3]:
positions = [
    Position(lookup.find_symbol('BINANCE', s), ZERO_COSTS) for s in ['BTCUSDT', 'ETHUSDT']
] 

positions[0].change_position_by(0, 0.05, 63000)
positions[1].change_position_by(0, 0.5, 3200)

0

In [11]:
import socket
print(socket.gethostname())

LAPTOP-O0L8A55Q


In [4]:
class PositionsWriter(LogsWriter):

    def write_data(self, log_type, data: List[Dict[str, Any]]):
        table = defaultdict(list)
        total_pnl, total_mkv = 0, 0
        
        for r in data:
            table['Symbol'].append(r['instrument_id'])
            table['Time'].append(r['timestamp'])
            table['Quantity'].append(r['quantity'])
            table['AvgPrice'].append(r['avg_position_price'])
            table['LastPrice'].append(r['current_price'])
            table['PnL'].append(r['pnl_quoted'])
            table['MarketValue'].append(r['market_value_quoted'])
            total_pnl += r['pnl_quoted']
            total_mkv += r['market_value_quoted']

        table['Symbol'].append('TOTAL')
        table['PnL'].append(total_pnl)
        table['MarketValue'].append(total_mkv)

        # - write to database table here
        print(f" ::: Strategy {self.strategy_id} @ {self.account_id} :::")
        print(tabulate.tabulate(table, ['Symbol', 'Time', 'Quantity', 'AvgPrice', 'LastPrice', 'PnL', 'MarketValue'], tablefmt='rounded_grid'))

In [6]:
pos_dumper = PositionsDumper(PositionsWriter('test', 'strategy1', '000'), '5Sec')
pos_dumper.attach_positions(*positions)

In [7]:
for _ in range(30):
    t = pd.Timestamp('now').asm8
    for p in positions:
        p.update_market_price(t, p.last_update_price + 10, 1)
    pos_dumper.store(t)
    time.sleep(0.5)

 ::: Strategy strategy1 @ test :::
╭──────────┬────────────────────────────┬────────────┬────────────┬─────────────┬───────┬───────────────╮
│ Symbol   │ Time                       │   Quantity │   AvgPrice │   LastPrice │   PnL │   MarketValue │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ BTCUSDT  │ 2024-04-23T16:58:19.311414 │       0.05 │      63000 │       63010 │   0.5 │        3150.5 │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ ETHUSDT  │ 2024-04-23T16:58:19.311414 │       0.5  │       3200 │        3210 │   5   │        1605   │
├──────────┼────────────────────────────┼────────────┼────────────┼─────────────┼───────┼───────────────┤
│ TOTAL    │                            │            │            │             │   5.5 │        4755.5 │
╰──────────┴────────────────────────────┴────────────┴────────────┴─────────────┴───────┴───────────────╯
 ::: Strate

In [ ]:
pos_dumper.store(pd.Timestamp('now').asm8)

In [4]:
pd.read_csv('logs/FlipFlopStrat_binance-mde_positions.csv', index_col=['instrument_id'], parse_dates=['timestamp'])

,timestamp,pnl_quoted,quantity,realized_pnl_quoted,avg_position_price,current_price,market_value_quoted,run_id
instrument_id,,,,,,,,
PEPEUSDT,2024-04-23 15:00:55.077420,0.011400,380000.00,0.0,0.000008,0.000008,2.918400,LAPTOP-O0L8A55Q-17138844309
SHIBUSDT,2024-04-23 15:00:55.077420,0.000000,0.00,0.0,0.000000,0.000027,0.000000,LAPTOP-O0L8A55Q-17138844309
XVGUSDT,2024-04-23 15:00:55.077420,0.000000,0.00,0.0,0.000000,0.006230,0.000000,LAPTOP-O0L8A55Q-17138844309
BOMEUSDT,2024-04-23 15:00:55.077420,-4.367286,3014.00,0.0,0.012421,0.010972,33.069608,LAPTOP-O0L8A55Q-17138844309
CKBUSDT,2024-04-23 15:00:55.077420,0.000000,0.00,0.0,0.000000,0.019174,0.000000,LAPTOP-O0L8A55Q-17138844309
GALAUSDT,2024-04-23 15:00:55.077420,-3.706560,429.00,0.0,0.058230,0.049590,21.274110,LAPTOP-O0L8A55Q-17138844309
LINKUSDT,2024-04-23 15:00:55.077420,-2.565050,1.45,0.0,17.150000,15.381000,22.302450,LAPTOP-O0L8A55Q-17138844309


In [5]:
pd.read_csv('logs/FlipFlopStrat_binance-mde_portfolio.csv', index_col=['timestamp'], parse_dates=['timestamp'])

,instrument_id,pnl_quoted,quantity,realized_pnl_quoted,avg_position_price,current_price,market_value_quoted,exchange_time,commissions_quoted,run_id
timestamp,,,,,,,,,,
2024-04-23 14:25:00,PEPEUSDT,-0.003800,380000.00,0.0,0.000008,0.000008,2.903200,2024-04-23T14:26:55.054067000,0.002907,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:25:00,SHIBUSDT,0.000000,0.00,0.0,0.000000,0.000027,0.000000,2024-04-23T14:26:55.054067000,0.000000,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:25:00,XVGUSDT,0.000000,0.00,0.0,0.000000,0.006225,0.000000,2024-04-23T14:26:55.054067000,0.000000,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:25:00,BOMEUSDT,-5.096674,3014.00,0.0,0.012421,0.010730,32.340220,2024-04-23T14:26:55.054067000,0.037439,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:25:00,CKBUSDT,0.000000,0.00,0.0,0.000000,0.019203,0.000000,2024-04-23T14:26:55.054067000,0.000000,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:25:00,GALAUSDT,-3.736590,429.00,0.0,0.058230,0.049520,21.244080,2024-04-23T14:26:55.054067000,0.024985,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:25:00,LINKUSDT,-2.534600,1.45,0.0,17.150000,15.402000,22.332900,2024-04-23T14:26:55.054067000,0.024868,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:30:00,PEPEUSDT,-0.003800,380000.00,0.0,0.000008,0.000008,2.903200,2024-04-23T14:30:55.446354000,0.002907,LAPTOP-O0L8A55Q-17138823653
2024-04-23 14:30:00,SHIBUSDT,0.000000,0.00,0.0,0.000000,0.000027,0.000000,2024-04-23T14:30:55.446354000,0.000000,LAPTOP-O0L8A55Q-17138823653


In [6]:
pd.read_csv('logs/FlipFlopStrat_binance-mde_executions.csv', index_col=['timestamp'], parse_dates=['timestamp'])

,instrument_id,side,filled_qty,price,commissions,commissions_quoted,run_id
timestamp,,,,,,,
2024-04-23 14:26:30.425,PEPEUSDT,buy,380000.0,0.000008,0.000003,BNB,LAPTOP-O0L8A55Q-17138823653


In [7]:
pd.read_csv('logs/FlipFlopStrat_binance-mde_balance.csv', index_col=['timestamp'], parse_dates=['timestamp'])

,instrument_id,total,locked,run_id
timestamp,,,,
2024-04-23 15:02:07.134389,BNB,0.047887,0.0,LAPTOP-O0L8A55Q-17138845238
2024-04-23 15:02:07.134389,USDT,60.669210,0.0,LAPTOP-O0L8A55Q-17138845238
2024-04-23 15:02:07.134389,LINK,1.450000,0.0,LAPTOP-O0L8A55Q-17138845238
2024-04-23 15:02:07.134389,GALA,429.000000,0.0,LAPTOP-O0L8A55Q-17138845238
2024-04-23 15:02:07.134389,ETHW,0.024330,0.0,LAPTOP-O0L8A55Q-17138845238
2024-04-23 15:02:07.134389,PEPE,380000.000000,0.0,LAPTOP-O0L8A55Q-17138845238
2024-04-23 15:02:07.134389,BOME,3014.000000,0.0,LAPTOP-O0L8A55Q-17138845238
